In [1]:
import numpy as np
import random

In [13]:
#Example universe

num_universes = 50
num_variables = 6
max_iterations = 1000
P_demand = 3678.15

lower_bound = np.array([400, 400, 200, 200, 100, 100])   # Lower bounds for each DER
upper_bound = np.array([800, 800, 600, 600, 400, 400]) # Upper bounds for each DER

alpha = np.array([0.001, 0.001, 0.002, 0.002, 0.005, 0.005])
beta = np.array([20, 20, 15, 15, 40, 40])
gamma = np.array([1000, 1000, 500, 500, 2000, 2000])


In [14]:
# Q-learning parameters
num_states = 100
num_actions = num_variables
learning_rate = 0.1
discount_factor = 0.9
epsilon = 1.0
epsilon_decay = 0.995
min_epsilon = 0.01

q_table_size = num_states ** num_variables
q_table = np.zeros((q_table_size, num_actions))

In [15]:
def fitness_function_with_penalty(universe, P_demand, penalty_factor=10000):
    P_total = np.sum(universe)

    original_cost = np.sum(alpha * universe**2 + beta * universe + gamma)

    penalty = penalty_factor * max(0, P_demand - P_total)

    penalized_cost = original_cost + penalty

    return penalized_cost


In [16]:
def discretize_state(universe, lower_bound, upper_bound, num_states):
    relative_state = (universe - lower_bound) / (upper_bound - lower_bound)
    discrete_state = (relative_state * (num_states - 1)).astype(int)
    flat_index = np.sum(discrete_state * (num_states ** np.arange(len(discrete_state))))
    max_index = num_states ** len(universe) - 1
    flat_index = min(flat_index, max_index)
    return flat_index

In [6]:
def initialize_universes(num_universes, num_variables, lower_bound, upper_bound):
    return np.random.uniform(low=lower_bound, high=upper_bound, size=(num_universes, num_variables))

universes = initialize_universes(num_universes, num_variables, lower_bound, upper_bound)


In [17]:
fitness = np.array([fitness_function_with_penalty(universe, P_demand) for universe in universes])


In [18]:
def white_black_hole_mechanism(universes, fitness):
    sorted_indices = np.argsort(fitness)
    best_universe = universes[sorted_indices[0]]
    worst_universe = universes[sorted_indices[-1]]

    for i in range(num_universes):
        if i != sorted_indices[0]:
            universes[i] += np.random.uniform(0, 1) * (best_universe - universes[i])
            universes[i] -= np.random.uniform(0, 1) * (universes[i] - worst_universe)

    return universes


In [19]:
def wormhole_mechanism(universes, q_table, iteration, max_iterations, lower_bound, upper_bound):
    WEP = 0.2 + iteration * (1.0 - 0.2) / max_iterations  # Wormhole Existence Probability
    TDR = 1 - (iteration / max_iterations)**6  # Traveling Distance Rate

    for i in range(num_universes):
        if np.random.uniform(0, 1) < WEP:
            state = discretize_state(universes[i], lower_bound, upper_bound, num_states)

            if np.random.uniform(0, 1) < epsilon:
                action = np.random.randint(0, num_actions)  # Explore
            else:
                action = np.argmax(q_table[state])  # Exploit

            adjustment = TDR * np.random.uniform(-1, 1) * (upper_bound[action] - lower_bound[action])
            universes[i][action] += adjustment

            universes[i] = np.clip(universes[i], lower_bound, upper_bound)

            new_state = discretize_state(universes[i], lower_bound, upper_bound, num_states)
            reward = -fitness_function_with_penalty(universes[i], P_demand)  # Reward is the negative cost

            old_value = q_table[state, action]
            next_max = np.max(q_table[new_state])
            q_table[state, action] = old_value + learning_rate * (reward + discount_factor * next_max - old_value)

    return universes


In [20]:
best_solutions = []

for iteration in range(max_iterations):
    universes = white_black_hole_mechanism(universes, fitness)

    universes = wormhole_mechanism(universes, q_table, iteration, max_iterations, lower_bound, upper_bound)


    fitness = np.array([fitness_function_with_penalty(universe, P_demand) for universe in universes])

    best_fitness_idx = np.argmin(fitness)
    best_solutions.append(universes[best_fitness_idx])

    if epsilon > min_epsilon:
        epsilon *= epsilon_decay

In [21]:
best_universe = min(best_solutions, key=lambda u: fitness_function_with_penalty(u, P_demand))
min_cost = fitness_function_with_penalty(best_universe, P_demand)

print("Best solution (power outputs):", best_universe)
print("Minimum generation cost:", min_cost)


Best solution (power outputs): [788.93818028 799.62847316 599.67988254 599.46827183 397.9113097
 387.97164417]
Minimum generation cost: 1137960.3190974272
